In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as n
from math import *
import datetime as dt
# from learnweight import ComputeWeight
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
import pyarrow.parquet as pq

## Generate trip dataset with time and area id. 

In [5]:
trips = pq.read_table('..//..//..//Database//NYC_trip//yellow_tripdata_2022-06.parquet')
trips = trips.to_pandas()
trips

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,1,2022-06-01 00:25:41,2022-06-01 00:48:22,1.0,11.00,1.0,N,70,48,1,32.00,3.00,0.5,2.00,6.55,0.3,44.35,2.5,0.00
1,1,2022-06-01 00:44:40,2022-06-01 01:01:48,1.0,4.20,1.0,N,170,226,1,14.00,3.00,0.5,0.00,0.00,0.3,17.80,2.5,0.00
2,2,2022-06-01 00:23:07,2022-06-01 00:39:50,1.0,9.49,1.0,N,264,113,1,26.00,0.50,0.5,5.00,6.55,0.3,42.60,2.5,1.25
3,1,2022-06-01 00:25:53,2022-06-01 00:57:06,2.0,12.10,1.0,N,132,17,2,37.00,1.75,0.5,0.00,0.00,0.3,39.55,0.0,1.25
4,1,2022-06-01 00:23:58,2022-06-01 00:33:43,0.0,1.80,1.0,N,140,163,1,9.00,3.00,0.5,2.55,0.00,0.3,15.35,2.5,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3558119,1,2022-06-30 23:45:51,2022-06-30 23:51:48,NaN,0.00,NaN,None,148,256,0,9.20,0.50,0.5,0.00,0.00,0.3,15.00,NaN,NaN
3558120,2,2022-06-30 23:25:00,2022-06-30 23:40:00,NaN,5.01,NaN,None,79,262,0,18.86,0.00,0.5,5.19,0.00,0.3,27.35,NaN,NaN
3558121,2,2022-06-30 23:29:00,2022-06-30 23:37:00,NaN,1.55,NaN,None,164,79,0,10.03,0.00,0.5,3.10,0.00,0.3,16.43,NaN,NaN
3558122,2,2022-06-30 23:24:15,2022-06-30 23:50:19,NaN,5.30,NaN,None,211,239,0,24.34,0.00,0.5,0.00,0.00,0.3,27.64,NaN,NaN


In [6]:
trips = trips[['PULocationID','tpep_pickup_datetime','DOLocationID','tpep_dropoff_datetime']]
trips.columns = ['sid','call_time','eid','end_time']

In [7]:
area = pd.read_csv("..//..//..//Database//NYC_area//NY_area_df.csv",index_col=0)
area.columns =  [int(float(s)) for s in area.columns]
trips = trips[trips['sid'].isin(area.columns)]
trips = trips[trips['eid'].isin(area.columns)]
trips.to_csv('..//..//..//Database//NYC_trip//order.csv')

In [8]:
np.random.seed(0)
driver0 = pd.DataFrame({'id':np.random.choice(area.columns , size=1000)})
driver0.to_csv('..//..//..//Database//NYC_trip//driver.csv')

In [9]:
trips[trips['sid'] ==44]

,sid,call_time,eid,end_time
10554,44,2022-06-01 07:38:00,140,2022-06-01 09:37:31
261024,44,2022-06-03 09:16:56,210,2022-06-03 09:54:40
343181,44,2022-06-03 21:18:07,210,2022-06-03 21:54:07
389684,44,2022-06-04 11:58:05,135,2022-06-04 13:06:19
468285,44,2022-06-04 22:20:57,210,2022-06-04 23:13:05
495398,44,2022-06-05 07:52:32,135,2022-06-05 08:44:10
550029,44,2022-06-05 18:15:37,210,2022-06-05 19:06:16
668429,44,2022-06-06 20:34:23,210,2022-06-06 21:07:18
918490,44,2022-06-08 21:12:39,210,2022-06-08 21:49:29
1175693,44,2022-06-10 22:10:48,210,2022-06-10 22:43:16


In [10]:
order = trips
order['call_time']=pd.to_datetime(order['call_time']); order['end_time']=pd.to_datetime(order['end_time'])
start_time = pd.to_datetime('2022-06-01 06:00:00 AM'); end_time = pd.to_datetime('2022-06-01 07:00:00 AM')
order_pick = order[(order['call_time'] > start_time) & (order['call_time'] <= end_time)]
order_pick = order_pick[['sid','call_time','eid','end_time']]
order_pick['sid'] -= 1
order_pick['eid'] -= 1
order_pick = order_pick[(order_pick['sid']<=262)&(order_pick['eid']<=262)]

In [11]:
order_pick.iloc[0,3]-order_pick.iloc[0,1]

Timedelta('0 days 00:08:01')

In [12]:
order_pick

,sid,call_time,eid,end_time
3676,262,2022-06-01 06:00:31,160,2022-06-01 06:08:32
3855,185,2022-06-01 06:00:17,160,2022-06-01 06:04:21
4174,131,2022-06-01 06:30:12,137,2022-06-01 06:57:39
4257,225,2022-06-01 06:56:30,225,2022-06-01 06:56:54
4258,232,2022-06-01 06:54:03,41,2022-06-01 07:06:01
...,...,...,...,...
3425956,237,2022-06-01 06:26:00,136,2022-06-01 06:50:00
3425957,157,2022-06-01 06:50:00,99,2022-06-01 06:58:00
3425958,67,2022-06-01 06:13:13,161,2022-06-01 06:22:21
3426019,47,2022-06-01 07:00:00,74,2022-06-01 07:14:00
